In [29]:
import pandas as pd 
import numpy as np 
from pathlib import Path

In [45]:
path = "../raw/AWS EC2 Carbon Footprint Dataset - EC2 Instances Dataset.csv"
df=pd.read_csv(path,decimal=',')

d={'instance': df["Instance type"], '0%':df["Instance @ Idle"],'10%':df["Instance @ 10%"],'50':df["Instance @ 50%"],'100%':df["Instance @ 100%"]}
slimdf=pd.DataFrame(data=d)
print(slimdf)
dfpath=Path(f"../processed/slim_data/ec2_instances.csv")
slimdf.to_csv(dfpath,index=False)

          instance    0%   10%    50  100%
0        a1.medium   1.2   1.9   3.2   4.2
1         a1.large   2.4   3.8   6.4   8.5
2        a1.xlarge   4.8   7.6  12.7  17.0
3       a1.2xlarge   9.5  15.2  25.4  34.0
4       a1.4xlarge  19.0  30.3  50.9  67.9
..             ...   ...   ...   ...   ...
616    db.t2.small   2.0   3.3   5.3   7.0
617   db.t2.medium   4.0   6.6  10.7  14.1
618    db.t2.large   4.8   7.8  12.3  16.5
619   db.t2.xlarge   9.6  15.7  24.6  33.0
620  db.t2.2xlarge  19.2  31.4  49.1  66.0

[621 rows x 5 columns]


In [46]:
x=[0,0.1,0.5,1]
for index,row in slimdf.iterrows():
    y=[row["0%"],row["10%"]]
    print(y)

[1.2, 1.9]
[2.4, 3.8]
[4.8, 7.6]
[9.5, 15.2]
[19.0, 30.3]
[19.0, 30.3]
[2.9, 4.8]
[11.6, 19.3]
[122.5, 198.4]
[85.8, 143.3]
[4.4, 7.4]
[8.9, 14.8]
[17.7, 29.5]
[35.5, 59.1]
[71.0, 118.2]
[5.1, 8.5]
[10.2, 16.9]
[20.3, 33.9]
[40.6, 67.8]
[89.9, 150.3]
[4.9, 7.8]
[9.7, 15.6]
[19.4, 31.3]
[38.8, 62.5]
[87.3, 140.7]
[95.2, 168.9]
[174.7, 281.4]
[190.5, 337.8]
[190.5, 337.8]
[2.6, 4.2]
[5.3, 8.4]
[10.6, 16.9]
[21.2, 33.8]
[42.3, 67.6]
[63.5, 101.4]
[84.6, 135.1]
[126.9, 202.7]
[2.6, 4.2]
[5.3, 8.4]
[10.6, 16.9]
[21.2, 33.8]
[42.3, 67.6]
[63.5, 101.4]
[84.6, 135.1]
[126.9, 202.7]
[4.9, 7.8]
[9.7, 15.6]
[19.4, 31.3]
[38.8, 62.5]
[87.3, 140.7]
[95.2, 168.9]
[174.7, 281.4]
[190.5, 337.8]
[190.5, 337.8]
[5.1, 8.3]
[10.2, 16.5]
[20.4, 33.0]
[40.8, 66.1]
[92.0, 149.2]
[184.0, 298.3]
[184.0, 298.3]
[1.1, 1.8]
[2.3, 3.6]
[4.6, 7.3]
[9.1, 14.5]
[18.3, 29.0]
[36.5, 58.1]
[54.8, 87.1]
[73.0, 116.1]
[73.0, 116.1]
[1.1, 1.8]
[2.3, 3.6]
[4.6, 7.3]
[9.1, 14.5]
[18.3, 29.0]
[36.5, 58.1]
[54.8, 87.1]
[73.0, 